# Github Repo Scraper

In [1]:
#import libraries
import os
import re
import bs4
import csv
import time
import requests
import json, codecs
from pprint import pprint
from github import Github
from bs4 import BeautifulSoup

In [2]:
#write results on a file in json format
def write_json(page_objects,page):
    
    path_name = '../data1'
    if not os.path.exists(path_name):
        os.makedirs(path_name)
    
    file = 'open_data_word_emb' + str(page) + '.json'
    
    with open(os.path.join(path_name,file),'a') as f:
        json.dump(page_objects,f)

In [3]:
#define functions 
def find_tags(tags,each_issue):
    for counter,temp in enumerate(each.find_all('a')):
        tag = str(temp.string)
        if tag != "None":
            tags.append(tag)
        
    remove_special_ch(tags)

In [4]:
#maybe embody it in the find_desc function to reduce iterations.
def remove_special_ch(text):
    
    for i in range(0,len(text)):
        text[i] = text[i].replace('\n',' ')
        text[i] = text[i].replace('\t',' ')
        text[i] = re.sub(r"<.*?>"," ",text[i])
        #maybe better solution is <[^<>]+> matches any character except < or > 
        #one or more times included inside < and > 

In [5]:
def search_all_page(stack_trace,page):
    temp = page.find_all('div',class_='edit-comment-hide')
    for i in temp:
        temp_2 = i.find('td')
        temp_desc = []
        calculate_desc(temp_2,temp_desc,stack_trace,0)
        if stack_trace != []:
            break

In [6]:
def find_desc(each,base_url,description,stack_trace,is_bug):
    
    next_url     = base_url+each.find('a')['href']
    response     = requests.get(next_url)
    html_content = response.content
    dom          = BeautifulSoup(html_content,'html.parser')
    temp         = dom.find('div',class_ = 'edit-comment-hide')
    temp_2       = temp.find('td')
    
    #find description and the stack trace
    calculate_desc(temp_2,description,stack_trace,1)
    
    #search all the page for a stack trace
    if stack_trace == [] and is_bug == 1:
        search_all_page(stack_trace,dom)
    
    #find the person who closed the issue
    who_closed_it = "none"
    temp_3        = dom.find_all('div',class_='TimelineItem-body') 
    for i in temp_3:
        if i.text.find('closed this')!=-1:
            if i.find('a',class_='author Link--primary text-bold') is None:
                continue
            who_closed_it = i.find('a',class_='author Link--primary text-bold').text
    
    #remove special characters from the description and stack trace
    remove_special_ch(description)
    
    if stack_trace != []:
        remove_special_ch(stack_trace)
    
    return who_closed_it

In [7]:
def calculate_desc(html_text,description,stack_trace,flag):
    
    html_content = html_text.contents
    length = len(html_content)
    
    if length == 1:
        #some lines are empty. No need to write them
        if html_content[0]!= []:
            str_temp = str(html_content[0])
            if str_temp.find('.java:')!=-1 or str_temp.find('java.')!=-1 or str_temp.find('AndroidRuntime:')!=-1:
                stack_trace.append(str(html_content[0]))
            else:
                if flag == 1:
                    if str(html_content[0]).startswith('<code>') != True:
                        description.append(str(html_content[0]))
    else:
        for i in range(length):
            
            if type(html_content[i]) is bs4.element.NavigableString:
                #avoid writting empty lines and html tags <br>
                if len(html_content[i])>4:
                        str_temp = str(html_content[i])
                        if str_temp.find('.java:')!=-1 or str_temp.find('java.')!=-1 or str_temp.find('AndroidRuntime:')!=-1:
                            stack_trace.append(str(html_content[i]))
                        elif flag == 1:
                            description.append(str(html_content[i]))
            elif type(html_content[i] is bs4.element.Tag):
                #call recursively the function till length is 1
                calculate_desc(html_content[i],description,stack_trace,flag)
        

In [8]:
#search all closed issues

#define some variables
total_issues  = 0
total_traces  = 0
total_bugs    = 0
pages_counter = 0
issues        = []


#initialize important url's
base_url = "https://github.com/"
#query_url = f"https://github.com/cgeo/cgeo/issues?page=1&q=is%3Aissue+is%3Aclosed"
query_url = f"https://github.com/cgeo/cgeo/issues?page=1&q=is%3Aissue+is%3Aopen"

#authorization in order to make more requests.
token = os.getenv("GITHUB_TOKEN")
headers = {'Authorization': f'token {token}'}

response = requests.get(query_url, headers = headers)

In [9]:
#loop through all pages

while True:
    response_code = response.status_code
    
    if response_code != 200:
        raise Exception("Error Occured")
    else:
        html_content = response.content
        dom = BeautifulSoup(html_content,'html.parser')
        
        #find all issues in every page
        all = dom.findAll('div', class_='flex-auto min-width-0 p-2 pr-3 pr-md-2')
        
        page_objects = []
        pages_counter += 1
        
        #real scraping begins
        #search all issues per page
        for each in all:
            
            #find tags and who open the issue
            tags=[]
            find_tags(tags,each)
            
            #flag that is activated if the issue is bug
            is_bug = 0
            for i in tags:
                if i == "Bug":
                    is_bug = 1
                    total_bugs = total_bugs+1
            
            #find description, stack trace and who closed
            description   = []
            stack_trace   = []
            who_closed_it = find_desc(each,base_url,description,stack_trace,is_bug)
            
            total_issues = total_issues+1
            if stack_trace !=[]:
                total_traces = total_traces +1
            
            print(total_issues,tags[0])
            
            #write dictionary
            if len(tags)>=1:
                issue_object = {'name':tags[0],'tags':tags[1:-1],'opened_by':tags[len(tags)-1],
                                'description':description,'stack_trace':stack_trace,'closed_by':who_closed_it}
            
            
            page_objects.append(issue_object)
        
        #write issues by page 
        write_json(page_objects,pages_counter)
        
        #combine all issues to a global list
        #issues.extend(page_objects)
        
        #visit the next page if exists
        end = dom.findAll('a',class_='next_page')
        if end == []:
            break
        
        next_url = base_url + end[1]['href']
        response = requests.get(next_url)
        
        ##for debugging
        if total_issues % 25 != 0:
            print("##########################\nTHE PAGE##############\n ",pages_counter,"HAS NOT 25 ISSUES",)
        ##
        
        # Sleep for 60 seconds
        time.sleep(60)

1 Advanced Filter - "Origin" - add all (active) entries automatic or/and add button "add all"
2 Advanced Filter - "Favorites" with too much space
3 Quick settings - reorder "Show Caches:" and use same wording in QS and Filter
4 Global search - set "Search with filter" at the first position
5 Use coordinate input dialog for distance filter
6 Option: After importing GPX, automatically update all caches
7 Search by address - result -> different caches when tap on text or map symbol
8 Filter: Long-click action on filter button to reset/restore filter
9 Filtering for attributes: exclude multiple attributes does not work
10 Lists -> Manage Caches -> Refresh Personal Notes
11 Dialog Header color inconsistent 
12 wrong icon color in context menu, when coordinates are selected
13 Filter: Let filters which include T5 also include >T5
14 Filter: make displayed filter string fully use available display space
15 Add "AND/OR" switch to status filter 
16 Add [does not contain] to text filters 
17 Add

144 Codacy/PMD.ExcessiveClassLength: too long class cgeo.geocaching.maps.mapsforge.v6.NewMap
145 Implement field notes formatted as UTF16-LE with BOM
146 Cleaning of deleted caches
147 Non-SDK API warning on Google Play
148 Handling of [NUMBER] in conjunction with disabling offline log display
149 add option: delete waypoint
150 Backup should include not-yet-send offline log images
151 Special Caches wrongly shown as archived
152 [Nightly] Swipe down for refresh does not work if there are no logs
153 Scrolling vs showing waypoint map at waypoint page
154 Add Map rotation for mapsforge maps
155 Merge "map theme" and "map style" into one dialog
156 OSM tile cache gets purged too frequently
157 tiles doesn't get repainted when switching from CyclOSM to combined offline map
158 Target SDK 30 (and use App bundle format) until November 2021
159 Ignore caches (for basic members + other connectors)
160 How to build from source
161 [route sort] Switch to RecyclerView
162 Increase the amount of 

303 Log (discover/grab) trackable produces error
304 Make live map disabled status transient
305 Allow user defined audio file/tones for proximity notification
306 New cache types from OCPL
307 GK Log shows date in the future
308 cache.logCounts is  lost when cache is cached
309 Offline log checkmark barely visible
310 TB inventory is blank after isActive() has been implemented in TravelBugConnector
311 c:geo cannot handle changing Geocode
312 Sync found status from live map to saved caches
313 Map zoom buttons z-order not ok
314 Search for trackable code working without active connectors
315 HTML background color within Listing not rendered 
316 Show account name (and avatar) in settings menu also for other connector
317 Limit number of caches for "show on map"
318 HTML-Anchor still not working
319 Harmonize maps - Support waypoint selector also on Google Maps
320 Make export functions cancellable
321 Check metadata of c:geo fdroid repo
322 Use blocks in description (when available)
3

476 Implement checker for challenge caches
477 Compass calibration notice
478 Allow to browse friends
479 Use Geocoder to get city of cache
480 More options for GPX import
481 Favorite percentage is not significant e.g. for GPX imported caches
482 Store trackables
483 per service signature
484 c:geo does insufficient cleanup
485 History should not include old finds, just recently found caches.
486 New design for detail activity
487 Add changelog to Crowdin
488 Feature Request: Transmit .gpx file instead of URL via NFC
489 Have some news inbox inside the app
490 Have dedicated github user for CI
491 Show affected caches when exporting field notes
492 GCVote shown as "Active" although no password given
493 Support for *.sqlitedb offline maps
494 Lock screen feature request
495 No option to load more caches (if ec.com is active)
496 Opt-in for loading cache log images immediately
497 Generalize pocket query lookup in connector
498 Request own logs and show on friends/own logs tab
499 Supp

In [10]:
#print some statistics
print(total_bugs)
print(total_traces)
print(total_issues)

0
28
563


In [ ]:
# to ask
# make small changes and code that is not stack trace i discard it. Is it ok??